In [1]:
!pip install category_encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import category_encoders as ce

In [3]:
df = pd.read_csv("/content/Loan_default.csv")
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [4]:
df.isnull().sum()

LoanID            0
Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          255347 non-null  object 
 1   Age             255347 non-null  int64  
 2   Income          255347 non-null  int64  
 3   LoanAmount      255347 non-null  int64  
 4   CreditScore     255347 non-null  int64  
 5   MonthsEmployed  255347 non-null  int64  
 6   NumCreditLines  255347 non-null  int64  
 7   InterestRate    255347 non-null  float64
 8   LoanTerm        255347 non-null  int64  
 9   DTIRatio        255347 non-null  float64
 10  Education       255347 non-null  object 
 11  EmploymentType  255347 non-null  object 
 12  MaritalStatus   255347 non-null  object 
 13  HasMortgage     255347 non-null  object 
 14  HasDependents   255347 non-null  object 
 15  LoanPurpose     255347 non-null  object 
 16  HasCoSigner     255347 non-null  object 
 17  Default   

In [6]:
print("rows: ",df.shape[0])
print("cols: ",df.shape[1])

rows:  255347
cols:  18


In [7]:
x = df.drop(columns=['LoanID', 'Default'])
y = df['Default']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [9]:
x_encoder = ce.OrdinalEncoder(cols=['Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage',
                                     'HasDependents', 'LoanPurpose', 'HasCoSigner'])

X_train=x_encoder.fit_transform(X_train)
X_test=x_encoder.transform(X_test)
y_encoder=ce.OrdinalEncoder(cols=['Default'])
y_train=y_encoder.fit_transform(y_train)
y_test=y_encoder.transform(y_test)
y_test = (y_test == 2).astype(int)
y_train = (y_train == 2).astype(int)



# Define the XGBoost parameters
params = {
    'objective': 'binary:logistic',  # Binary classification problem
    'eval_metric': 'logloss',         # Logarithmic Loss
    'max_depth': 3,                   # Maximum depth of the tree
    'eta': 0.1,                       # Learning rate
    'subsample': 0.75,                # Fraction of samples used for training
    'colsample_bytree': 0.8,          # Fraction of features used for training
    'min_child_weight': 1,            # Minimum sum of instance weight (hessian) needed in a child
    'gamma': 0,                       # Minimum loss reduction required to make a further partition
    'seed': 42                        # Random seed for reproducibility
}


In [10]:
# Fit the XGBoost model
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate classification report
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(class_report)

Accuracy: 0.8863198458574181
Confusion Matrix:
[[56243   167]
 [ 7090   337]]

Classification Report:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     56410
           1       0.67      0.05      0.08      7427

    accuracy                           0.89     63837
   macro avg       0.78      0.52      0.51     63837
weighted avg       0.86      0.89      0.84     63837



In [11]:
from sklearn.metrics import roc_auc_score

# Make predictions on the test data
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.7550218605930197
